ASAP CRN Metadata validation - wave 2

# Team Sulzer. ASAP CRN Metadata validation - wave 2

13 Oct 2024
Andy Henrie




In [1]:
import pandas as pd
from pathlib import Path
import os, sys

sys.path.append(os.path.abspath((os.path.join(os.getcwd(), 'src/crn_utils'))))

from util import read_CDE, NULL, prep_table, read_meta_table
from validate import validate_table, ReportCollector
from update_schema import v1_to_v2, v2_to_v3_PMDBS, create_upload_medadata_package

%load_ext autoreload
%autoreload 2


Streamlit NOT successfully imported


In [2]:
# !gsutil -u dnastack-asap-parkinsons cp "gs://asap-raw-data-team-sulzer/manifest.txt" ./sulzer_manifest_11-22-2023.txt


## CDEs
load the relavent CDEs

In [3]:
schema_version = "v1"
schema_path = Path.home() / "Projects/ASAP/crn-utils/resource/CDE"
CDEv1 = read_CDE(schema_version, local_path=schema_path)
schema_version = "v2.1"
CDEv2 = read_CDE(schema_version, local_path=schema_path)
schema_version = "v3.0"
CDEv3 = read_CDE(schema_version, local_path=schema_path)

metadata_version: ASAP_CDE_v1
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v1
/Users/ergonyc/Projects/ASAP/crn-utils/resource/CDE/ASAP_CDE_v1.csv
read local file
metadata_version: ASAP_CDE_v2.1
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v2.1
/Users/ergonyc/Projects/ASAP/crn-utils/resource/CDE/ASAP_CDE_v2.1.csv
read local file
metadata_version: ASAP_CDE_v3.0
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v3.0
/Users/ergonyc/Projects/ASAP/crn-utils/resource/CDE/ASAP_CDE_v3.0.csv
read local file


> SANITY CHECK: verify reading from google doc works.

```python
CDEv1_ = read_CDE("v1")
CDEv2_ = read_CDE("v2.1")
CDEv3_ = read_CDE("v3.0")
```


## Clean V1 Table
write clean metadata tables according to CDE v1

### Team Sulzer

In [4]:
## convert 
team = "sulzer"
collection = "sn_pmdbs"

root_path = Path.home() / ("Projects/ASAP/data/teams")

metadata_path = root_path / f"{team}/{collection}/metadata/"
og_path = metadata_path / "og"

metadata_version = "v2"

METADATA_VERSION_DATE = f"{metadata_version}_{pd.Timestamp.now().strftime('%Y%m%d')}"

In [5]:
CDE = CDEv2
tables = CDE['Table'].unique()

dfs = {}
for table in tables:
    df = read_meta_table(og_path / f"{table}.csv")
    schema = CDE[CDE['Table'] == table]
    report = ReportCollector(destination="NA")
    full_table, report = validate_table(df.copy(), table, schema, report)
    report.print_log()
    dfs[table] = full_table
    # df.to_csv(v1_path / f"{table}.csv", index=False)

recoding number_of_brain_samples as int
🚨⚠️❗ **Missing Required Fields in STUDY: submitter_email**
🚨⚠️❗ **Missing Optional Fields in STUDY: alternate_dataset_id**
🚨⚠️❗ **8 Fields with empty (NULL) values:**

	- ASAP_grant_id: 1/1 empty rows (REQUIRED)

	- other_funding_source: 1/1 empty rows (REQUIRED)

	- publication_DOI: 1/1 empty rows (REQUIRED)

	- publication_PMID: 1/1 empty rows (REQUIRED)

	- PI_ORCHID: 1/1 empty rows (OPTIONAL)

	- PI_google_scholar_id: 1/1 empty rows (OPTIONAL)

	- preprocessing_references: 1/1 empty rows (OPTIONAL)

	- metadata_version_date: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.
🚨⚠️❗ **Extra field in STUDY: submittor_email**

All required fields are present in *PROTOCOL* table.
🚨⚠️❗ **3 Fields with empty (NULL) values:**

	- github_url: 1/1 empty rows (REQUIRED)

	- protocols_io_DOI: 1/1 empty rows (REQUIRED)

	- other_reference: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

recoding age_at_onset as int
recod

In [6]:

SUBJECT = read_meta_table(og_path / "SUBJECT.csv")
CLINPATH = read_meta_table(og_path / "CLINPATH.csv")
STUDY = read_meta_table(og_path / "STUDY.csv")
PROTOCOL = read_meta_table(og_path / "PROTOCOL.csv")
SAMPLE = read_meta_table(og_path / "SAMPLE.csv")
DATA = read_meta_table(og_path / "DATA.csv")



In [7]:
STUDY['metadata_version_date'] = METADATA_VERSION_DATE
STUDY.rename(columns={"submittor_email":"submitter_email"}, inplace=True)
STUDY['alternate_dataset_id'] = NULL
STUDY

,ASAP_team_name,ASAP_lab_name,project_name,team_dataset_id,project_dataset,project_description,PI_full_name,PI_email,contributor_names,submitter_name,...,publication_PMID,number_of_brain_samples,brain_regions,types_of_samples,PI_ORCHID,PI_google_scholar_id,DUA_version,preprocessing_references,metadata_version_date,alternate_dataset_id
0,TEAM-SULZER,Al-Dalahmah,The landscape of T-cell Immunity in Parkinson ...,Dalahmah_Tcells,OJK_2022_ASAP_SEQ,Assessing the role of T cells in Parkinson's d...,Osama Al-Dalahmah,oa2298@cumc.columbia.edu,"Kelly A Jakubiak, Adithya Kannan, Jaeseung Lee...",Kelly Jakubiak,...,NA,44,Cingulate Cortex and Substantia nigra,Postmortem PD and control,NA,NA,v1,NA,v2_20241016,NA


In [8]:

SUBJECT = prep_table(SUBJECT, CDE)
SUBJECT['sex'] = SUBJECT['sex'].replace({"M":"Male", "F":"Female"})
SUBJECT.head()


,subject_id,source_subject_id,AMPPD_id,GP2_id,biobank_name,organism,sex,age_at_collection,race,ethnicity,...,hx_dementia_mci,hx_melanoma,education_level,smoking_status,smoking_years,APOE_e4_status,cognitive_status,time_from_baseline,primary_diagnosis,primary_diagnosis_text
0,OJ9,T-4530,NA,NA,New York Brain Bank,Human,Male,77,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
1,OJ8,T-4105,NA,NA,New York Brain Bank,Human,Female,89+,Unknown,Hispanic or Latino,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
2,OJ7,T-5259,NA,NA,New York Brain Bank,Human,Male,79,White,Not Hispanic or Latino,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
3,OJ6,T-5385,NA,NA,New York Brain Bank,Human,Male,88,White,Not Hispanic or Latino,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
4,OJ5,T-2465,NA,NA,New York Brain Bank,Human,Female,83,Black or African American,Not Hispanic or Latino,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown


In [9]:
SUBJECT['subject_id'].shape


(46,)

In [10]:
SUBJECT['source_subject_id'].unique()


array(['T-4530', 'T-4105', 'T-5259', 'T-5385', 'T-2465', 'T-3799',
       'T-4424', 'T-5133', 'T-5227', 'T-5547', 'T-5359', 'T-4116',
       'T-4839', 'T-2007', 'T-4938', 'T-5498', 'T-2791', 'T-4695',
       'T-4934', 'T-3870', 'T-124', 'T-81', 'T-4515', 'T-4235', 'T-4815',
       'T-4857', 'T-3998', 'T-4462', 'T-4108', 'T-4174', 'T3860',
       'T-4162', 'T-4210', 'T-233', 'T1976', 'T-2017', 'T-308', 'T-1952',
       'T-777', 'T-1619', 'T290', 'T-551', 'T-257', 'T-343', 'T-362'],
      dtype=object)

In [11]:


SUBJECT['hx_dementia_mci'] = SUBJECT['hx_dementia_mci'].replace("Unknown", NULL)

SUBJECT['hx_melanoma'] = SUBJECT['hx_melanoma'].replace("Unknown", NULL)
SUBJECT['education_level'] = SUBJECT['education_level'].replace("Unknown", NULL)
SUBJECT['cognitive_status'] = SUBJECT['cognitive_status'].replace("Unknown", NULL)
SUBJECT['primary_diagnosis'] = SUBJECT['primary_diagnosis'].replace("Unknown", NULL)


In [12]:
SAMPLE = prep_table(SAMPLE, CDE)

SAMPLE["organism_ontology_term_id"] = "NCBITaxon:9606"
SAMPLE['molecular_source'] = SAMPLE['molecular_source'].replace("RNA", "PolyA RNA")
SAMPLE['region_level_2'] = SAMPLE['region_level_2'].replace('Substantia Nigra', 'Substantia nigra',)
SAMPLE.head()

,sample_id,subject_id,source_sample_id,replicate,replicate_count,repeated_sample,batch,tissue,brain_region,hemisphere,...,sex_ontology_term_id,self_reported_ethnicity_ontology_term_id,disease_ontology_term_id,tissue_ontology_term_id,cell_type_ontology_term_id,assay_ontology_term_id,suspension_type,DV200,pm_PH,donor_id
0,T-362,OF001,T-362,Rep1,1,0,1,Brain,Cingulate Cortex,NA,...,PATO:0000383 (female),Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA
1,T-308,OF009,T-308,Rep1,1,0,1,Brain,Cingulate Cortex,NA,...,PATO:0000384 (male),Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA
2,T3860,OF016,T3860,Rep1,1,0,1,Brain,Cingulate Cortex,NA,...,PATO:0000384 (male),Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA
3,T290,OF005,T290,Rep1,1,0,1,Brain,Cingulate Cortex,NA,...,PATO:0000384 (male),Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA
4,T-233,OF013,T-233,Rep1,1,0,1,Brain,Cingulate Cortex,NA,...,PATO:0000384 (male),Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA


In [13]:
fix_samples = {'T3860':'T-3860', 'T290':'T-290','T1976':'T-1976'}
SAMPLE['sample_id'] = SAMPLE['sample_id'].replace(fix_samples)
SAMPLE['source_sample_id'] = SAMPLE['source_sample_id'].replace(fix_samples)
SAMPLE.head()

,sample_id,subject_id,source_sample_id,replicate,replicate_count,repeated_sample,batch,tissue,brain_region,hemisphere,...,sex_ontology_term_id,self_reported_ethnicity_ontology_term_id,disease_ontology_term_id,tissue_ontology_term_id,cell_type_ontology_term_id,assay_ontology_term_id,suspension_type,DV200,pm_PH,donor_id
0,T-362,OF001,T-362,Rep1,1,0,1,Brain,Cingulate Cortex,NA,...,PATO:0000383 (female),Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA
1,T-308,OF009,T-308,Rep1,1,0,1,Brain,Cingulate Cortex,NA,...,PATO:0000384 (male),Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA
2,T-3860,OF016,T-3860,Rep1,1,0,1,Brain,Cingulate Cortex,NA,...,PATO:0000384 (male),Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA
3,T-290,OF005,T-290,Rep1,1,0,1,Brain,Cingulate Cortex,NA,...,PATO:0000384 (male),Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA
4,T-233,OF013,T-233,Rep1,1,0,1,Brain,Cingulate Cortex,NA,...,PATO:0000384 (male),Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA


In [14]:
batch_mapper = {"Cingulate Cortex":"cing1", "Substantia Nigra": "sn1"}
SAMPLE['batch']=SAMPLE['brain_region'].map(batch_mapper)

## 🚨⚠️❗ EXTREME CURATION HERE 🚨⚠️❗
The codeing by Team-Sulzer is inconsistent.  However I am able to infer the following:

TODO:  Define `sample_id`s from `subject_id`, and create `sample_id` as `source_subject_id`


`source_subject_id`s seem to be unique and correct.  Although the CLINPATH.duration_pmi are different for `source_subject_id`="T-2465", and some demographic info is missing for the cingulate sample, these seem to be two samples from the same subject.  

Hence:  We will assume `source_subject_id` is ground truth.   Redefine `sample_id` from the current `subject_id` (which matches file naming conventsions), and  create a _new_ `subject_id` from the `source_subject_id` by prepending "SUL_".  Finally, `source_sample_id` will be set to pd.NA as is currently just a copy of `source_subject_id`

### fix SAMPLE table first

In [15]:
# simple copy subject_id to sample_id, sample_id == source_sample_id
SAMPLE['sample_id'] = SAMPLE['subject_id']


In [16]:
fix_samples = {'T3860':'T-3860', 'T290':'T-290','T1976':'T-1976'}
SAMPLE['source_sample_id'] = SAMPLE['source_sample_id'].replace(fix_samples)
# then make a subject_id by copying souce_sample_id
SAMPLE['subject_id'] = "SUL_" + SAMPLE['source_sample_id'].str.replace("-","_")

# # finally set source_sample_id to pd.NA
SAMPLE['source_sample_id'] = NULL

SAMPLE['alternate_sample_id'] = NULL

SAMPLE.head()

,sample_id,subject_id,source_sample_id,replicate,replicate_count,repeated_sample,batch,tissue,brain_region,hemisphere,...,self_reported_ethnicity_ontology_term_id,disease_ontology_term_id,tissue_ontology_term_id,cell_type_ontology_term_id,assay_ontology_term_id,suspension_type,DV200,pm_PH,donor_id,alternate_sample_id
0,OF001,SUL_T_362,NA,Rep1,1,0,cing1,Brain,Cingulate Cortex,NA,...,Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA,NA
1,OF009,SUL_T_308,NA,Rep1,1,0,cing1,Brain,Cingulate Cortex,NA,...,Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA,NA
2,OF016,SUL_T_3860,NA,Rep1,1,0,cing1,Brain,Cingulate Cortex,NA,...,Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA,NA
3,OF005,SUL_T_290,NA,Rep1,1,0,cing1,Brain,Cingulate Cortex,NA,...,Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA,NA
4,OF013,SUL_T_233,NA,Rep1,1,0,cing1,Brain,Cingulate Cortex,NA,...,Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA,NA


### now SUBJECT table 

In [17]:
fix_samples = {'T3860':'T-3860', 'T290':'T-290','T1976':'T-1976'}
SUBJECT['source_subject_id'] = SUBJECT['source_subject_id'].replace(fix_samples)

SUBJECT['subject_id'] = "SUL_" + SUBJECT['source_subject_id'].str.replace("-","_")
SUBJECT['subject_id'].unique()

array(['SUL_T_4530', 'SUL_T_4105', 'SUL_T_5259', 'SUL_T_5385',
       'SUL_T_2465', 'SUL_T_3799', 'SUL_T_4424', 'SUL_T_5133',
       'SUL_T_5227', 'SUL_T_5547', 'SUL_T_5359', 'SUL_T_4116',
       'SUL_T_4839', 'SUL_T_2007', 'SUL_T_4938', 'SUL_T_5498',
       'SUL_T_2791', 'SUL_T_4695', 'SUL_T_4934', 'SUL_T_3870',
       'SUL_T_124', 'SUL_T_81', 'SUL_T_4515', 'SUL_T_4235', 'SUL_T_4815',
       'SUL_T_4857', 'SUL_T_3998', 'SUL_T_4462', 'SUL_T_4108',
       'SUL_T_4174', 'SUL_T_3860', 'SUL_T_4162', 'SUL_T_4210',
       'SUL_T_233', 'SUL_T_1976', 'SUL_T_2017', 'SUL_T_308', 'SUL_T_1952',
       'SUL_T_777', 'SUL_T_1619', 'SUL_T_290', 'SUL_T_551', 'SUL_T_257',
       'SUL_T_343', 'SUL_T_362'], dtype=object)

### and CLINPATH table is already coded as it should be



In [18]:

fix_samples = {'T3860':'T-3860', 'T290':'T-290','T1976':'T-1976'}
CLINPATH['source_subject_id'] = CLINPATH['source_subject_id'].replace(fix_samples)
CLINPATH['source_subject_id'].unique()

array(['T-4530', 'T-4105', 'T-5259', 'T-5385', 'T-2465', 'T-3799',
       'T-4424', 'T-5133', 'T-5227', 'T-5547', 'T-5359', 'T-4116',
       'T-4839', 'T-2007', 'T-4938', 'T-5498', 'T-2791', 'T-4695',
       'T-4934', 'T-3870', 'T-124', 'T-81', 'T-4515', 'T-4235', 'T-4815',
       'T-4857', 'T-3998', 'T-4462', 'T-4108', 'T-4174', 'T-3860',
       'T-4162', 'T-4210', 'T-233', 'T-1976', 'T-2017', 'T-308', 'T-1952',
       'T-777', 'T-1619', 'T-290', 'T-551', 'T-257', 'T-343', 'T-362'],
      dtype=object)

In [19]:
CLINPATH['subject_id'] = "SUL_" + CLINPATH['source_subject_id'].str.replace("-","_")
CLINPATH.head()

,subject_id,source_subject_id,duration_pmi,path_autopsy_dx_main,path_autopsy_second_dx,path_autopsy_third_dx,path_autopsy_fourth_dx,path_autopsy_fifth_dx,path_autopsy_sixth_dx,path_autopsy_seventh_dx,...,path_nia_ri,path_nia_aa_a,path_nia_aa_b,path_nia_aa_c,TDP43,arteriolosclerosis_severity_scale,amyloid_angiopathy_severity_scale,path_ad_level,dig_slide_avail,quant_path_avail
0,SUL_T_4530,T-4530,17.06666667,Parkinson's disease with dementia,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,No,No
1,SUL_T_4105,T-4105,20,Alzheimer's disease (intermediate level neurop...,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,No,No
2,SUL_T_5259,T-5259,10.5,Parkinson's disease with dementia,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,No,No
3,SUL_T_5385,T-5385,14.05,"Control, Primary age-related tauopathy (PART)",NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,No,No
4,SUL_T_2465,T-2465,9.25,"Control, Low level AD neuropathological change",NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,No,No


## DATA table

The DATA.csv was not filled out.  So we will construct it by joining the manifest from the source of data transfer for Team Sulzer.

### read_file_list
Helper function to read sulzer manifest 

In [20]:

cingulate_files = Path.home() / "Projects/ASAP/team-sulzer/cingulate_manifest.txt"
nigra_files = Path.home() / "Projects/ASAP/team-sulzer/nigra_manifest.txt"

In [21]:
# tmp2 = pd.read_csv("sulzer_fastqs.txt", delimiter=" ", header=None, names=['file_name'])
def read_gz_file_list(file_nm):
    # Read the text file
    with open(file_nm, 'r') as file:
        lines = [line.strip() for line in file]

    # Create a DataFrame
    df = pd.DataFrame(lines, columns=['file_name'])

    df['parts'] = df['file_name'].apply(lambda x: x.lstrip("gs://").split("/"))
    df['source'] = df['parts'].apply(lambda x: x[1].lstrip("fastqs_") )
    df['name'] = df['parts'].apply(lambda x: x[-1])

    df['name_parts'] = df['name'].apply(lambda x: x.rstrip('.fastq.gz').split("_"))
    df['sample_id'] = df['name_parts'].apply(lambda x: x[0])
    df['lane_id'] = df['name_parts'].apply(lambda x: x[2])
    df['file_type'] = df['name_parts'].apply(lambda x: x[3])
    df.drop(columns=['parts','name_parts'], inplace=True)
    return df

# tmp2 = pd.read_csv("sulzer_fastqs.txt", delimiter=" ", header=None, names=['file_name'])
def read_file_list_sulzer(file_nm):
    # Read the text file
    keep_ends = ("fastq.gz", "bam", "tar")
    with open(file_nm, 'r') as file:
        lines = [line.strip() for line in file if line.strip().endswith(keep_ends)]

    # Create a DataFrame
    df = pd.DataFrame(lines, columns=['file_name'])

    df['parts'] = df['file_name'].apply(lambda x: x.split("/"))
    # df['source'] = df['parts'].apply(lambda x: x[5].lstrip("fastqs_") )
    df['file_name'] = df['parts'].apply(lambda x: x[-1])
    df['sample_id'] = df['parts'].apply(lambda x: x[6] if x[-1].endswith(("fastq.gz","tar")) else x[7])
    df['file_type'] = df['parts'].apply(lambda x: x[7] if x[7].strip() in ["fastq","analysis"] else "bam")
    # df['tmp1'] = df['parts'].apply(lambda x: x[7] if len(x) > 7 else pd.NA)

    # df['name_parts'] = df['name'].apply(lambda x: x.rstrip('.fastq.gz').split("_"))
    # df['sample_id'] = df['name_parts'].apply(lambda x: x[0])
    # df['lane_id'] = df['name_parts'].apply(lambda x: x[2])
    # df['file_type'] = df['name_parts'].apply(lambda x: x[3])
    df.drop(columns=['parts'], inplace=True)
    return df




In [22]:

c_fastqs = read_file_list_sulzer(cingulate_files)
c_fastqs['src'] = "cingulate"
n_fastqs = read_file_list_sulzer(nigra_files)
n_fastqs['src'] = "nigra"

from_source = pd.concat([c_fastqs, n_fastqs])

In [23]:
n_fastqs

,file_name,sample_id,file_type,src
0,OJ1_cellranger_count_outs.tar,OJ1,analysis,nigra
1,OJ1_S1_L001_I1_001.fastq.gz,OJ1,fastq,nigra
2,OJ1_S1_L001_I2_001.fastq.gz,OJ1,fastq,nigra
3,OJ1_S1_L001_R1_001.fastq.gz,OJ1,fastq,nigra
4,OJ1_S1_L001_R2_001.fastq.gz,OJ1,fastq,nigra
...,...,...,...,...
454,OJ9_S1_L003_R2_001.fastq.gz,OJ9,fastq,nigra
455,OJ9_S1_L004_I1_001.fastq.gz,OJ9,fastq,nigra
456,OJ9_S1_L004_I2_001.fastq.gz,OJ9,fastq,nigra
457,OJ9_S1_L004_R1_001.fastq.gz,OJ9,fastq,nigra


In [24]:
## now make DATA from scratch to ready to join into
# Start with sample_id
# DATA_ = read_meta_table(metadata_path / "DATA.csv")

DATA = SAMPLE[["sample_id", "replicate", "replicate_count","repeated_sample","batch"]].copy()

DATA.head()



,sample_id,replicate,replicate_count,repeated_sample,batch
0,OF001,Rep1,1,0,cing1
1,OF009,Rep1,1,0,cing1
2,OF016,Rep1,1,0,cing1
3,OF005,Rep1,1,0,cing1
4,OF013,Rep1,1,0,cing1


In [25]:
DATA = pd.merge(DATA, from_source, on='sample_id', how='left')
DATA.tail()

,sample_id,replicate,replicate_count,repeated_sample,batch,file_name,file_type,src
707,OJ26,Rep1,1,0,sn1,OJ26_S1_L003_R2_001.fastq.gz,fastq,nigra
708,OJ26,Rep1,1,0,sn1,OJ26_S1_L004_I1_001.fastq.gz,fastq,nigra
709,OJ26,Rep1,1,0,sn1,OJ26_S1_L004_I2_001.fastq.gz,fastq,nigra
710,OJ26,Rep1,1,0,sn1,OJ26_S1_L004_R1_001.fastq.gz,fastq,nigra
711,OJ26,Rep1,1,0,sn1,OJ26_S1_L004_R2_001.fastq.gz,fastq,nigra


In [26]:
DATA['omic'] = 'RNA'
DATA['adjustments'] = 'Raw'
DATA['technology'] = "SN"
DATA['file_description']= DATA['file_type'].map({"bam": "Binary aligned sequence",
                                                "fastq": "gzip fastq",
                                                "analysis": "cellranger counts tar archive"})
DATA['file_description']= DATA['file_type'].map({"bam": "Reads",
                                                "fastq": "Reads",
                                                "analysis": "Counts"})

# now fix the file_type
DATA['file_type'] = DATA['file_type'].replace({'fastq':'fastq', "bam":'Per sample raw file', "analysis":'Per sample processed file'})
DATA['content'] = 'Counts'



get "md5s"

In [27]:


cingulate_md5s = Path.home() / "Projects/ASAP/team-sulzer/cingulate_fastq_md5.log"
nigra_md5s = Path.home() / "Projects/ASAP/team-sulzer/nigra_fastq_md5.log"

In [28]:
from checksums import extract_hashes_from_gsutil

# cingulate
cingulate_hashs = extract_hashes_from_gsutil(cingulate_md5s)

# nigra
nigra_hashs = extract_hashes_from_gsutil(nigra_md5s)

# combine cingulate and nigra
source_file_crc = cingulate_hashs[0] | nigra_hashs[0]
source_file_md5 = cingulate_hashs[1] | nigra_hashs[1]


In [29]:
DATA['file_MD5'] = DATA['file_name'].map(source_file_md5)

In [30]:

DATA.head()



,sample_id,replicate,replicate_count,repeated_sample,batch,file_name,file_type,src,omic,adjustments,technology,file_description,content,file_MD5
0,OF001,Rep1,1,0,cing1,possorted_genome_bam.bam,Per sample raw file,cingulate,RNA,Raw,SN,Reads,Counts,NaN
1,OF001,Rep1,1,0,cing1,OF001_cellranger_count_outs.tar,Per sample processed file,cingulate,RNA,Raw,SN,Counts,Counts,NaN
2,OF001,Rep1,1,0,cing1,OF001_S1_L001_I1_001.fastq.gz,fastq,cingulate,RNA,Raw,SN,Reads,Counts,74a74c8929b01a6da7e79b83f91a6607
3,OF001,Rep1,1,0,cing1,OF001_S1_L001_I2_001.fastq.gz,fastq,cingulate,RNA,Raw,SN,Reads,Counts,644d95728b50f1ff9db7becfa5b5c394
4,OF001,Rep1,1,0,cing1,OF001_S1_L001_R1_001.fastq.gz,fastq,cingulate,RNA,Raw,SN,Reads,Counts,129dfb6fe6115966e2cf2af5e3c82bdf


# No data found for `OJ8`!!!

# load manifests and md5s

Separate manifests for cingulate and nigra to make it easier for Team Sulzer to collect.


In [31]:
SAMPLE[SAMPLE['sample_id']=='OJ8']
empty_sample = SAMPLE['sample_id']=='OJ8'
print(SAMPLE.shape)
SAMPLE = SAMPLE[~empty_sample]
SAMPLE.shape

(46, 34)


(45, 34)

In [32]:
DATA[DATA['sample_id']=='OJ8']
empty_sample = DATA['sample_id']=='OJ8'
print(DATA.shape)
DATA = DATA[~empty_sample]
DATA.shape

(712, 14)


(711, 14)

In [33]:
DATA.rename(columns={'adjustments':'adjustment'}, inplace=True)
DATA['time'] = NULL
DATA['header'] = NULL
DATA['annotation'] = NULL
DATA['configuration_file'] = NULL

DATA.head()

,sample_id,replicate,replicate_count,repeated_sample,batch,file_name,file_type,src,omic,adjustment,technology,file_description,content,file_MD5,time,header,annotation,configuration_file
0,OF001,Rep1,1,0,cing1,possorted_genome_bam.bam,Per sample raw file,cingulate,RNA,Raw,SN,Reads,Counts,NaN,NA,NA,NA,NA
1,OF001,Rep1,1,0,cing1,OF001_cellranger_count_outs.tar,Per sample processed file,cingulate,RNA,Raw,SN,Counts,Counts,NaN,NA,NA,NA,NA
2,OF001,Rep1,1,0,cing1,OF001_S1_L001_I1_001.fastq.gz,fastq,cingulate,RNA,Raw,SN,Reads,Counts,74a74c8929b01a6da7e79b83f91a6607,NA,NA,NA,NA
3,OF001,Rep1,1,0,cing1,OF001_S1_L001_I2_001.fastq.gz,fastq,cingulate,RNA,Raw,SN,Reads,Counts,644d95728b50f1ff9db7becfa5b5c394,NA,NA,NA,NA
4,OF001,Rep1,1,0,cing1,OF001_S1_L001_R1_001.fastq.gz,fastq,cingulate,RNA,Raw,SN,Reads,Counts,129dfb6fe6115966e2cf2af5e3c82bdf,NA,NA,NA,NA


In [34]:
SUBJECT['subject_id'].unique()

array(['SUL_T_4530', 'SUL_T_4105', 'SUL_T_5259', 'SUL_T_5385',
       'SUL_T_2465', 'SUL_T_3799', 'SUL_T_4424', 'SUL_T_5133',
       'SUL_T_5227', 'SUL_T_5547', 'SUL_T_5359', 'SUL_T_4116',
       'SUL_T_4839', 'SUL_T_2007', 'SUL_T_4938', 'SUL_T_5498',
       'SUL_T_2791', 'SUL_T_4695', 'SUL_T_4934', 'SUL_T_3870',
       'SUL_T_124', 'SUL_T_81', 'SUL_T_4515', 'SUL_T_4235', 'SUL_T_4815',
       'SUL_T_4857', 'SUL_T_3998', 'SUL_T_4462', 'SUL_T_4108',
       'SUL_T_4174', 'SUL_T_3860', 'SUL_T_4162', 'SUL_T_4210',
       'SUL_T_233', 'SUL_T_1976', 'SUL_T_2017', 'SUL_T_308', 'SUL_T_1952',
       'SUL_T_777', 'SUL_T_1619', 'SUL_T_290', 'SUL_T_551', 'SUL_T_257',
       'SUL_T_343', 'SUL_T_362'], dtype=object)

In [35]:
CLINPATH['subject_id'].unique()

array(['SUL_T_4530', 'SUL_T_4105', 'SUL_T_5259', 'SUL_T_5385',
       'SUL_T_2465', 'SUL_T_3799', 'SUL_T_4424', 'SUL_T_5133',
       'SUL_T_5227', 'SUL_T_5547', 'SUL_T_5359', 'SUL_T_4116',
       'SUL_T_4839', 'SUL_T_2007', 'SUL_T_4938', 'SUL_T_5498',
       'SUL_T_2791', 'SUL_T_4695', 'SUL_T_4934', 'SUL_T_3870',
       'SUL_T_124', 'SUL_T_81', 'SUL_T_4515', 'SUL_T_4235', 'SUL_T_4815',
       'SUL_T_4857', 'SUL_T_3998', 'SUL_T_4462', 'SUL_T_4108',
       'SUL_T_4174', 'SUL_T_3860', 'SUL_T_4162', 'SUL_T_4210',
       'SUL_T_233', 'SUL_T_1976', 'SUL_T_2017', 'SUL_T_308', 'SUL_T_1952',
       'SUL_T_777', 'SUL_T_1619', 'SUL_T_290', 'SUL_T_551', 'SUL_T_257',
       'SUL_T_343', 'SUL_T_362'], dtype=object)

In [36]:
SUBJECT['subject_id'].unique()
# CLINPATH.head()

array(['SUL_T_4530', 'SUL_T_4105', 'SUL_T_5259', 'SUL_T_5385',
       'SUL_T_2465', 'SUL_T_3799', 'SUL_T_4424', 'SUL_T_5133',
       'SUL_T_5227', 'SUL_T_5547', 'SUL_T_5359', 'SUL_T_4116',
       'SUL_T_4839', 'SUL_T_2007', 'SUL_T_4938', 'SUL_T_5498',
       'SUL_T_2791', 'SUL_T_4695', 'SUL_T_4934', 'SUL_T_3870',
       'SUL_T_124', 'SUL_T_81', 'SUL_T_4515', 'SUL_T_4235', 'SUL_T_4815',
       'SUL_T_4857', 'SUL_T_3998', 'SUL_T_4462', 'SUL_T_4108',
       'SUL_T_4174', 'SUL_T_3860', 'SUL_T_4162', 'SUL_T_4210',
       'SUL_T_233', 'SUL_T_1976', 'SUL_T_2017', 'SUL_T_308', 'SUL_T_1952',
       'SUL_T_777', 'SUL_T_1619', 'SUL_T_290', 'SUL_T_551', 'SUL_T_257',
       'SUL_T_343', 'SUL_T_362'], dtype=object)

In [37]:
SAMPLE

,sample_id,subject_id,source_sample_id,replicate,replicate_count,repeated_sample,batch,tissue,brain_region,hemisphere,...,self_reported_ethnicity_ontology_term_id,disease_ontology_term_id,tissue_ontology_term_id,cell_type_ontology_term_id,assay_ontology_term_id,suspension_type,DV200,pm_PH,donor_id,alternate_sample_id
0,OF001,SUL_T_362,NA,Rep1,1,0,cing1,Brain,Cingulate Cortex,NA,...,Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA,NA
1,OF009,SUL_T_308,NA,Rep1,1,0,cing1,Brain,Cingulate Cortex,NA,...,Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA,NA
2,OF016,SUL_T_3860,NA,Rep1,1,0,cing1,Brain,Cingulate Cortex,NA,...,Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA,NA
3,OF005,SUL_T_290,NA,Rep1,1,0,cing1,Brain,Cingulate Cortex,NA,...,Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA,NA
4,OF013,SUL_T_233,NA,Rep1,1,0,cing1,Brain,Cingulate Cortex,NA,...,Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA,NA
5,OF018,SUL_T_4108,NA,Rep1,1,0,cing1,Brain,Cingulate Cortex,NA,...,Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA,NA
6,OF003,SUL_T_257,NA,Rep1,1,0,cing1,Brain,Cingulate Cortex,NA,...,Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA,NA
7,OF007,SUL_T_777,NA,Rep1,1,0,cing1,Brain,Cingulate Cortex,NA,...,Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA,NA
8,OF011,SUL_T_1976,NA,Rep1,1,0,cing1,Brain,Cingulate Cortex,NA,...,Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA,NA
9,OF015,SUL_T_4162,NA,Rep1,1,0,cing1,Brain,Cingulate Cortex,NA,...,Unknown,NA,UBERON:0003027,NA,NA,Nucleus,NA,NA,NA,NA


In [38]:
dict(zip(CLINPATH['subject_id'], CLINPATH['path_autopsy_dx_main']))

{'SUL_T_4530': "Parkinson's disease with dementia",
 'SUL_T_4105': "Alzheimer's disease (intermediate level neuropathological change)",
 'SUL_T_5259': "Parkinson's disease with dementia",
 'SUL_T_5385': 'Control, Primary age-related tauopathy (PART)',
 'SUL_T_2465': 'Control, Low level AD neuropathological change',
 'SUL_T_3799': 'Other neurological disorder',
 'SUL_T_4424': "Parkinson's disease with dementia",
 'SUL_T_5133': "Alzheimer's disease (intermediate level neuropathological change)",
 'SUL_T_5227': "Alzheimer's disease (intermediate level neuropathological change)",
 'SUL_T_5547': 'Control, Primary age-related tauopathy (PART)',
 'SUL_T_5359': "Alzheimer's disease (intermediate level neuropathological change)",
 'SUL_T_4116': "Alzheimer's disease (intermediate level neuropathological change)",
 'SUL_T_4839': "Parkinson's disease with dementia",
 'SUL_T_2007': "Parkinson's disease with dementia",
 'SUL_T_4938': "Parkinson's disease with dementia",
 'SUL_T_5498': "Parkinson's d

In [39]:
SUBJECT['primary_diagnosis'] = SUBJECT['subject_id'].map(dict(zip(CLINPATH['subject_id'], CLINPATH['path_autopsy_dx_main'])))
SUBJECT['primary_diagnosis']

0                     Parkinson's disease with dementia
1     Alzheimer's disease (intermediate level neurop...
2                     Parkinson's disease with dementia
3         Control, Primary age-related tauopathy (PART)
4        Control, Low level AD neuropathological change
5                           Other neurological disorder
6                     Parkinson's disease with dementia
7     Alzheimer's disease (intermediate level neurop...
8     Alzheimer's disease (intermediate level neurop...
9         Control, Primary age-related tauopathy (PART)
10    Alzheimer's disease (intermediate level neurop...
11    Alzheimer's disease (intermediate level neurop...
12                    Parkinson's disease with dementia
13                    Parkinson's disease with dementia
14                    Parkinson's disease with dementia
15                    Parkinson's disease with dementia
16       Control, Low level AD neuropathological change
17                    Parkinson's disease with d

In [40]:
# via chatGPT
diagnosis_mapping = {
    "Parkinson's disease with dementia": "Dementia with Lewy bodies",
    "Alzheimer's disease (intermediate level neuropathological change)": "Alzheimer's disease",
    "Control, Primary age-related tauopathy (PART)": "Other neurological disorder",
    "Control, Low level AD neuropathological change": "Prodromal non-motor PD",
    "Control, Limbic predominant age-related TDP43 proteinopathy (LATE)": "Frontotemporal dementia",
    "Parkinson's disease": "Idiopathic PD",
    "Control, no misfolded protein or significant vascular pathology": "No PD nor other neurological disorder"
}


SUBJECT['primary_diagnosis'] = SUBJECT['primary_diagnosis'].replace(diagnosis_mapping)

In [41]:
v2_path = metadata_path / "v2"
STUDY.to_csv(v2_path / f"STUDY.csv", index=False)
SAMPLE.to_csv(v2_path / f"SAMPLE.csv", index=False)
SUBJECT.to_csv(v2_path / f"SUBJECT.csv", index=False)
PROTOCOL.to_csv(v2_path / f"PROTOCOL.csv", index=False)
CLINPATH.to_csv(v2_path / f"CLINPATH.csv", index=False)
DATA.to_csv(v2_path / f"DATA.csv", index=False)

In [42]:
CDE = CDEv2
tables = CDE['Table'].unique()
dfs = {}
for table in tables:
    schema = CDE[CDE['Table'] == table]
    df = read_meta_table(v2_path / f"{table}.csv")
    report = ReportCollector(destination="NA")
    full_table, report = validate_table(df.copy(), table, schema, report)
    dfs[table] = full_table
    report.print_log()

recoding number_of_brain_samples as int
All required fields are present in *STUDY* table.
🚨⚠️❗ **8 Fields with empty (NULL) values:**

	- ASAP_grant_id: 1/1 empty rows (REQUIRED)

	- other_funding_source: 1/1 empty rows (REQUIRED)

	- publication_DOI: 1/1 empty rows (REQUIRED)

	- publication_PMID: 1/1 empty rows (REQUIRED)

	- PI_ORCHID: 1/1 empty rows (OPTIONAL)

	- PI_google_scholar_id: 1/1 empty rows (OPTIONAL)

	- preprocessing_references: 1/1 empty rows (OPTIONAL)

	- alternate_dataset_id: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

All required fields are present in *PROTOCOL* table.
🚨⚠️❗ **3 Fields with empty (NULL) values:**

	- github_url: 1/1 empty rows (REQUIRED)

	- protocols_io_DOI: 1/1 empty rows (REQUIRED)

	- other_reference: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

recoding age_at_onset as int
recoding age_at_diagnosis as int
recoding first_motor_symptom as int
All required fields are present in *SUBJECT* table.
🚨⚠️❗ 

In [43]:
tables

array(['STUDY', 'PROTOCOL', 'SUBJECT', 'CLINPATH', 'SAMPLE', 'DATA'],
      dtype=object)

In [44]:
CDE = CDEv2
tables = CDE['Table'].unique()

dfs = {}
for table in tables:
    df = read_meta_table(v2_path / f"{table}.csv")
    schema = CDE[CDE['Table'] == table]

    valid_fields = schema['Field'].unique()
    df_out = df[valid_fields]
    aux_fields = set(df.columns) - set(valid_fields)
    if aux_fields:
        df_aux = df[list(aux_fields)]
        df_aux.to_csv(v2_path / f"{table}_auxiliary.csv", index=False)
        print(f"Saved {table}_auxiliary.csv")
    df_out.to_csv(v2_path / f"{table}.csv", index=False)
    dfs[table] = df_out

Saved DATA_auxiliary.csv


In [45]:
# make tables conform to CDE and save extra columns as "auxiliary"
v2_path = metadata_path / "v2"

for table in tables:
    df = dfs[table]
    schema = CDE[CDE['Table'] == table]
    valid_fields = schema['Field'].unique()
    df_out = df[valid_fields]
    aux_fields = set(df.columns) - set(valid_fields)
    if aux_fields:
        df_aux = df[list(aux_fields)]
        df_aux.to_csv(v2_path / f"{table}_auxiliary.csv", index=False)
        print(f"Saved {table}_auxiliary.csv")
    df_out.to_csv(v2_path / f"{table}.csv", index=False)

validate v2 tables


In [46]:
CDE = CDEv2
v2_tables = dfs
for table,df in v2_tables.items():
    schema = CDE[CDE['Table'] == table]

    report = ReportCollector(destination="NA")
    full_table, report = validate_table(df.copy(), table, schema, report)
    report.print_log()


recoding number_of_brain_samples as int
All required fields are present in *STUDY* table.
🚨⚠️❗ **8 Fields with empty (NULL) values:**

	- ASAP_grant_id: 1/1 empty rows (REQUIRED)

	- other_funding_source: 1/1 empty rows (REQUIRED)

	- publication_DOI: 1/1 empty rows (REQUIRED)

	- publication_PMID: 1/1 empty rows (REQUIRED)

	- PI_ORCHID: 1/1 empty rows (OPTIONAL)

	- PI_google_scholar_id: 1/1 empty rows (OPTIONAL)

	- preprocessing_references: 1/1 empty rows (OPTIONAL)

	- alternate_dataset_id: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

All required fields are present in *PROTOCOL* table.
🚨⚠️❗ **3 Fields with empty (NULL) values:**

	- github_url: 1/1 empty rows (REQUIRED)

	- protocols_io_DOI: 1/1 empty rows (REQUIRED)

	- other_reference: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

recoding age_at_onset as int
recoding age_at_diagnosis as int
recoding first_motor_symptom as int
All required fields are present in *SUBJECT* table.
🚨⚠️❗ 

In [47]:
v2_tables["SUBJECT"][['subject_id','primary_diagnosis'] ]
    # primary_diagnosis = v3_tables["SUBJECT"]['primary_diagnosis'].str.lower().str.replace(" ", "_")
    # diagnosis_mapper = dict(zip(subject_id, primary_diagnosis))
v2_tables["SUBJECT"]['subject_id'].unique()

array(['SUL_T_4530', 'SUL_T_4105', 'SUL_T_5259', 'SUL_T_5385',
       'SUL_T_2465', 'SUL_T_3799', 'SUL_T_4424', 'SUL_T_5133',
       'SUL_T_5227', 'SUL_T_5547', 'SUL_T_5359', 'SUL_T_4116',
       'SUL_T_4839', 'SUL_T_2007', 'SUL_T_4938', 'SUL_T_5498',
       'SUL_T_2791', 'SUL_T_4695', 'SUL_T_4934', 'SUL_T_3870',
       'SUL_T_124', 'SUL_T_81', 'SUL_T_4515', 'SUL_T_4235', 'SUL_T_4815',
       'SUL_T_4857', 'SUL_T_3998', 'SUL_T_4462', 'SUL_T_4108',
       'SUL_T_4174', 'SUL_T_3860', 'SUL_T_4162', 'SUL_T_4210',
       'SUL_T_233', 'SUL_T_1976', 'SUL_T_2017', 'SUL_T_308', 'SUL_T_1952',
       'SUL_T_777', 'SUL_T_1619', 'SUL_T_290', 'SUL_T_551', 'SUL_T_257',
       'SUL_T_343', 'SUL_T_362'], dtype=object)

In [48]:
v2_tables["SAMPLE"]['subject_id'].unique()

array(['SUL_T_362', 'SUL_T_308', 'SUL_T_3860', 'SUL_T_290', 'SUL_T_233',
       'SUL_T_4108', 'SUL_T_257', 'SUL_T_777', 'SUL_T_1976', 'SUL_T_4162',
       'SUL_T_4174', 'SUL_T_343', 'SUL_T_551', 'SUL_T_1619', 'SUL_T_1952',
       'SUL_T_2017', 'SUL_T_2465', 'SUL_T_4210', 'SUL_T_81', 'SUL_T_124',
       'SUL_T_2007', 'SUL_T_2791', 'SUL_T_3799', 'SUL_T_3870',
       'SUL_T_3998', 'SUL_T_4116', 'SUL_T_4235', 'SUL_T_4424',
       'SUL_T_4462', 'SUL_T_4515', 'SUL_T_4530', 'SUL_T_4695',
       'SUL_T_4815', 'SUL_T_4839', 'SUL_T_4857', 'SUL_T_4934',
       'SUL_T_4938', 'SUL_T_5133', 'SUL_T_5227', 'SUL_T_5259',
       'SUL_T_5359', 'SUL_T_5385', 'SUL_T_5498', 'SUL_T_5547'],
      dtype=object)

In [49]:
v3_meta_tables = ['STUDY', 'PROTOCOL', 'SUBJECT', 'SAMPLE', 'DATA', 'CLINPATH', 'PMDBS', 'CONDITION', 'ASSAY_RNAseq']

f"{v3_meta_tables}"

"['STUDY', 'PROTOCOL', 'SUBJECT', 'SAMPLE', 'DATA', 'CLINPATH', 'PMDBS', 'CONDITION', 'ASSAY_RNAseq']"

In [50]:
v3_path = metadata_path / "v3"

v3_tables, aux_tables = v2_to_v3_PMDBS(v2_path, v3_path, CDEv2, CDEv3)

recoding number_of_brain_samples as int
recoding age_at_onset as int
recoding age_at_diagnosis as int
recoding first_motor_symptom as int
recoding replicate_count as int
recoding repeated_sample as int
recoding input_cell_count as int
recoding replicate_count as int
recoding repeated_sample as int


In [51]:
CDE = CDEv3
for table,df in v3_tables.items():
    schema = CDE[CDE['Table'] == table]

    report = ReportCollector(destination="NA")
    full_table, report = validate_table(df.copy(), table, schema, report)
    report.print_log()

recoding number_samples as int
All required fields are present in *STUDY* table.
🚨⚠️❗ **8 Fields with empty (NULL) values:**

	- ASAP_grant_id: 1/1 empty rows (REQUIRED)

	- other_funding_source: 1/1 empty rows (REQUIRED)

	- publication_DOI: 1/1 empty rows (REQUIRED)

	- publication_PMID: 1/1 empty rows (REQUIRED)

	- PI_ORCID: 1/1 empty rows (OPTIONAL)

	- PI_google_scholar_id: 1/1 empty rows (OPTIONAL)

	- preprocessing_references: 1/1 empty rows (OPTIONAL)

	- alternate_dataset_id: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

All required fields are present in *PROTOCOL* table.
🚨⚠️❗ **3 Fields with empty (NULL) values:**

	- github_url: 1/1 empty rows (REQUIRED)

	- protocols_io_DOI: 1/1 empty rows (REQUIRED)

	- other_reference: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

All required fields are present in *SUBJECT* table.
🚨⚠️❗ **2 Fields with empty (NULL) values:**

	- race: 28/46 empty rows (REQUIRED)

	- primary_diagnosis_text: 46/

In [52]:
STUDY = v3_tables['STUDY']
STUDY

,ASAP_team_name,ASAP_lab_name,project_name,team_dataset_id,project_dataset,project_description,PI_full_name,PI_email,contributor_names,submitter_name,...,number_samples,sample_types,types_of_samples,DUA_version,metadata_tables,PI_ORCID,PI_google_scholar_id,preprocessing_references,metadata_version_date,alternate_dataset_id
0,TEAM-SULZER,Al-Dalahmah,The landscape of T-cell Immunity in Parkinson ...,Dalahmah_Tcells,OJK_2022_ASAP_SEQ,Assessing the role of T cells in Parkinson's d...,Osama Al-Dalahmah,oa2298@cumc.columbia.edu,"Kelly A Jakubiak, Adithya Kannan, Jaeseung Lee...",Kelly Jakubiak,...,44,Cingulate Cortex and Substantia nigra,Postmortem PD and control,v1,"['STUDY', 'PROTOCOL', 'SUBJECT', 'SAMPLE', 'DA...",NA,NA,NA,v2_20241016,NA


### Create metadata package


In [53]:
create_upload_medadata_package(metadata_path, v3_tables)

In [54]:
v3_tables['SAMPLE'][['sample_id','condition_id','subject_id']]


,sample_id,condition_id,subject_id
0,OF001,dementia_with_lewy_bodies,SUL_T_362
1,OF009,dementia_with_lewy_bodies,SUL_T_308
2,OF016,dementia_with_lewy_bodies,SUL_T_3860
3,OF005,dementia_with_lewy_bodies,SUL_T_290
4,OF013,dementia_with_lewy_bodies,SUL_T_233
5,OF018,dementia_with_lewy_bodies,SUL_T_4108
6,OF003,dementia_with_lewy_bodies,SUL_T_257
7,OF007,dementia_with_lewy_bodies,SUL_T_777
8,OF011,dementia_with_lewy_bodies,SUL_T_1976
9,OF015,dementia_with_lewy_bodies,SUL_T_4162


In [55]:
v3_tables['CONDITION']

Field,condition_id,intervention_name,intervention_id,protocol_id,intervention_aux_table
0,dementia_with_lewy_bodies,Case-Control,Other,NaN,NaN
1,alzheimer's_disease,Case-Control,Other,NaN,NaN
2,other_neurological_disorder,Case-Control,Other,NaN,NaN
3,prodromal_non-motor_pd,Case-Control,Other,NaN,NaN
4,frontotemporal_dementia,Case-Control,Other,NaN,NaN
5,idiopathic_pd,Case-Control,Case,NaN,NaN
6,no_pd_nor_other_neurological_disorder,Case-Control,Control,NaN,NaN


In [56]:
SUBJECT['subject_id'].shape


(46,)

## check remote files



In [31]:
!gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/sulzer-credentials.json 


Activated service account credentials for: [raw-admin-sulzer@dnastack-asap-parkinsons.iam.gserviceaccount.com]


In [29]:

!gsutil -u dnastack-asap-parkinsons ls -al "gs://asap-raw-data-team-sulzer/fastqs_cingulate_updated/**/*.gz"


   4249181  2023-12-01T21:26:39Z  gs://asap-raw-data-team-sulzer/fastqs_cingulate_updated/OF001/analysis/201123_OSAMA_FATIMA_2_HUMAN_10X-OF001-cellranger-count-default/OF001_cellranger_count_info.tar.gz#1701465999745881  metageneration=1
 752394240  2023-12-02T00:16:13Z  gs://asap-raw-data-team-sulzer/fastqs_cingulate_updated/OF001/fastq/SN_3PGEX_10X/210210_A00521_0199_BHNMK3DSXY/OF001_S1_L001_I1_001.fastq.gz#1701476173757016  metageneration=1
 757577390  2023-12-02T00:24:37Z  gs://asap-raw-data-team-sulzer/fastqs_cingulate_updated/OF001/fastq/SN_3PGEX_10X/210210_A00521_0199_BHNMK3DSXY/OF001_S1_L001_I2_001.fastq.gz#1701476677261783  metageneration=1
7398635734  2023-12-02T18:37:54Z  gs://asap-raw-data-team-sulzer/fastqs_cingulate_updated/OF001/fastq/SN_3PGEX_10X/210210_A00521_0199_BHNMK3DSXY/OF001_S1_L001_R1_001.fastq.gz#1701542274674731  metageneration=1
5587519431  2023-12-02T15:13:17Z  gs://asap-raw-data-team-sulzer/fastqs_cingulate_updated/OF001/fastq/SN_3PGEX_10X/210210_A00521_019

## fix missing file_name, file_description, file_MD5 

Collect "destination" or bucket md5s and manifest

In [74]:

!gsutil -u dnastack-asap-parkinsons ls "gs://asap-raw-data-team-sulzer/**/*fastq.gz" > sulzer_bucket_fasqs.txt

In [76]:
!gsutil -u dnastack-asap-parkinsons hash -h "gs://asap-raw-data-team-sulzer/**/*.gz" > sulzer_bucket_hexhash.log


using the module's C extension, so checksumming will run very slowly. For help
installing the extension, please see "gsutil help crcmod".



In [96]:
# bucket_fastqs = pd.read_csv("sulzer_bucket_fasqs.txt", delimiter="  ", header=None, names=['file_name'])
# bucket_fastqs

In [33]:
bucket_files_md5 = extract_md5_from_details2("sulzer_bucket_hexhash.log")


In [34]:
tmp = pd.DataFrame.from_dict(bucket_files_md5, orient="index",columns=['md5'])
tmp['file'] = tmp.index
tmp = tmp.sort_values(by='file').reset_index()

tmp


,index,md5,file
0,OF001_S1_L001_I1_001.fastq.gz,74a74c8929b01a6da7e79b83f91a6607,OF001_S1_L001_I1_001.fastq.gz
1,OF001_S1_L001_I2_001.fastq.gz,644d95728b50f1ff9db7becfa5b5c394,OF001_S1_L001_I2_001.fastq.gz
2,OF001_S1_L001_R1_001.fastq.gz,129dfb6fe6115966e2cf2af5e3c82bdf,OF001_S1_L001_R1_001.fastq.gz
3,OF001_S1_L001_R2_001.fastq.gz,dbdec2b83d4564e67284a51c8d1f5dc2,OF001_S1_L001_R2_001.fastq.gz
4,OF001_S1_L002_I1_001.fastq.gz,50d1c4be0b4c65310fac646fd676f3c4,OF001_S1_L002_I1_001.fastq.gz
...,...,...,...
724,OJ9_S1_L004_I1_001.fastq.gz,2dba0b38f76ba6490260266e02689392,OJ9_S1_L004_I1_001.fastq.gz
725,OJ9_S1_L004_I2_001.fastq.gz,ed6433715131d4ea289a5644bb89ce0b,OJ9_S1_L004_I2_001.fastq.gz
726,OJ9_S1_L004_R1_001.fastq.gz,a6ec79d470af33aab7e72c6703ea26fa,OJ9_S1_L004_R1_001.fastq.gz
727,OJ9_S1_L004_R2_001.fastq.gz,a9cb6810affb5ae38115d980048c31f9,OJ9_S1_L004_R2_001.fastq.gz


## Only check the fastq.gz files.  don't have md5s for the others

In [35]:
DATA_ = DATA[DATA['file_type']=="fastq"]

In [36]:



checksum = DATA_[['file_name','file_MD5']]
checksum['check1'] = checksum['file_MD5'].str.strip()
checksum['check2'] = checksum['file_name'].map(bucket_files_md5)



/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_40441/2423319654.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checksum['check1'] = checksum['file_MD5'].str.strip()
/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_40441/2423319654.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checksum['check2'] = checksum['file_name'].map(bucket_files_md5)


In [37]:
checksum[checksum.check1 != checksum.check2].file_name.to_list()


[]

In [38]:
checksum.head()

,file_name,file_MD5,check1,check2
2,OF001_S1_L001_I1_001.fastq.gz,74a74c8929b01a6da7e79b83f91a6607,74a74c8929b01a6da7e79b83f91a6607,74a74c8929b01a6da7e79b83f91a6607
3,OF001_S1_L001_I2_001.fastq.gz,644d95728b50f1ff9db7becfa5b5c394,644d95728b50f1ff9db7becfa5b5c394,644d95728b50f1ff9db7becfa5b5c394
4,OF001_S1_L001_R1_001.fastq.gz,129dfb6fe6115966e2cf2af5e3c82bdf,129dfb6fe6115966e2cf2af5e3c82bdf,129dfb6fe6115966e2cf2af5e3c82bdf
5,OF001_S1_L001_R2_001.fastq.gz,dbdec2b83d4564e67284a51c8d1f5dc2,dbdec2b83d4564e67284a51c8d1f5dc2,dbdec2b83d4564e67284a51c8d1f5dc2
6,OF001_S1_L002_I1_001.fastq.gz,50d1c4be0b4c65310fac646fd676f3c4,50d1c4be0b4c65310fac646fd676f3c4,50d1c4be0b4c65310fac646fd676f3c4


## Sanitize NULLs

In [41]:

CLINPATH = sanitize(CLINPATH)
PROTOCOL = sanitize(PROTOCOL)
SUBJECT = sanitize(SUBJECT)
STUDY = sanitize(STUDY)
DATA = sanitize(DATA)
SAMPLE = sanitize(SAMPLE)

## Export to clean

In [42]:

export_root = Path.cwd() / "clean/team-Sulzer"
if not export_root.exists():
    export_root.mkdir(parents=True, exist_ok=True)

STUDY.to_csv( export_root / "STUDY.csv")
PROTOCOL.to_csv(export_root / "PROTOCOL.csv")
SAMPLE.to_csv(export_root / "SAMPLE.csv")
SUBJECT.to_csv(export_root / "SUBJECT.csv")
CLINPATH.to_csv(export_root / "CLINPATH.csv")
DATA.to_csv(export_root / "DATA.csv")


## transfer metadata Sulzer 

In [43]:
# Sulzer
!gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/sulzer-credentials.json 


Activated service account credentials for: [raw-admin-sulzer@dnastack-asap-parkinsons.iam.gserviceaccount.com]


In [44]:

!gsutil -u dnastack-asap-parkinsons cp "./clean/team-sulzer/*.csv"  "gs://asap-raw-data-team-sulzer/metadata/v2/"

Copying file://./clean/team-sulzer/CLINPATH.csv [Content-Type=text/csv]...
Copying file://./clean/team-sulzer/SUBJECT.csv [Content-Type=text/csv]...       
Copying file://./clean/team-sulzer/SAMPLE.csv [Content-Type=text/csv]...        
Copying file://./clean/team-sulzer/DATA.csv [Content-Type=text/csv]...          
- [4 files][107.8 KiB/107.8 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://./clean/team-sulzer/STUDY.csv [Content-Type=text/csv]...
Copying file://./clean/team-sulzer/PROTOCOL.csv [Content-Type=text/csv]...      
\ [6 files][109.1 KiB/109.1 KiB]                                                
Operation completed over 6 objects/109.1 KiB.                                    


### copy metadata to workflow-dev bucket

First copy each set of metadata locally ...

In [65]:
!gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/wf-credentials.json

Activated service account credentials for: [admin-workflow-dev@dnastack-asap-parkinsons.iam.gserviceaccount.com]


In [66]:
!gsutil cp "./clean/team-Sulzer/*.csv" "gs://asap-workflow-dev/metadata/v2/sulzer"

Copying file://./clean/team-Sulzer/CLINPATH.csv [Content-Type=text/csv]...
Copying file://./clean/team-Sulzer/SUBJECT.csv [Content-Type=text/csv]...       
Copying file://./clean/team-Sulzer/SAMPLE.csv [Content-Type=text/csv]...        
Copying file://./clean/team-Sulzer/DATA.csv [Content-Type=text/csv]...          
- [4 files][112.1 KiB/112.1 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://./clean/team-Sulzer/STUDY.csv [Content-Type=text/csv]...
Copying file://./clean/team-Sulzer/PROTOCOL.csv [Content-Type=text/csv]...      
\ [6 files][113.5 KiB/113.5 KiB]                                                
Operation completed over 6 objects/113.5 KiB.                                    


In [67]:
!gsutil ls -al "gs://asap-workflow-dev/metadata/v2"

                                 gs://asap-workflow-dev/metadata/v2/hafler/
                                 gs://asap-workflow-dev/metadata/v2/hardy/
                                 gs://asap-workflow-dev/metadata/v2/jakobsson/
                                 gs://asap-workflow-dev/metadata/v2/lee/
                                 gs://asap-workflow-dev/metadata/v2/scherzer/
                                 gs://asap-workflow-dev/metadata/v2/sulzer/


In [312]:

# move to cannonical location
!gsutil -u dnastack-asap-parkinsons cp -r "gs://asap-raw-data-team-sulzer/fastqs_cingulate_updated" "gs://asap-raw-data-team-sulzer/fastqs/cingulate"


Copying gs://asap-raw-data-team-sulzer/fastqs_cingulate_updated/OF001/analysis/201123_OSAMA_FATIMA_2_HUMAN_10X-OF001-cellranger-count-default/OF001_cellranger_count_info.tar.gz [Content-Type=application/x-tar]...
Copying gs://asap-raw-data-team-sulzer/fastqs_cingulate_updated/OF001/analysis/201123_OSAMA_FATIMA_2_HUMAN_10X-OF001-cellranger-count-default/OF001_cellranger_count_outs.tar [Content-Type=application/x-tar]...
Copying gs://asap-raw-data-team-sulzer/fastqs_cingulate_updated/OF001/analysis/201123_OSAMA_FATIMA_2_HUMAN_10X-OF001-cellranger-count-default/README.txt [Content-Type=text/plain]...
Copying gs://asap-raw-data-team-sulzer/fastqs_cingulate_updated/OF001/analysis/201123_OSAMA_FATIMA_2_HUMAN_10X-OF001-cellranger-count-default/index.html [Content-Type=text/html]...
- [4 files][  1.2 GiB/  1.2 GiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Pl

In [317]:

# move to cannonical location
!gsutil -u dnastack-asap-parkinsons mv -r "gs://asap-raw-data-team-sulzer/fastqs_nigra" "gs://asap-raw-data-team-sulzer/fastqs/nigra"


Copying gs://asap-raw-data-team-sulzer/fastqs_nigra/OJ1/analysis/220224_OSAMA_JUNCHENG_8_HUMAN_10X-OJ1-cellranger-count-default/OJ1_cellranger_count_info.tar.gz [Content-Type=application/x-tar]...
Removing gs://asap-raw-data-team-sulzer/fastqs_nigra/OJ1/analysis/220224_OSAMA_JUNCHENG_8_HUMAN_10X-OJ1-cellranger-count-default/OJ1_cellranger_count_info.tar.gz...
Copying gs://asap-raw-data-team-sulzer/fastqs_nigra/OJ1/analysis/220224_OSAMA_JUNCHENG_8_HUMAN_10X-OJ1-cellranger-count-default/OJ1_cellranger_count_outs.tar [Content-Type=application/x-tar]...
Removing gs://asap-raw-data-team-sulzer/fastqs_nigra/OJ1/analysis/220224_OSAMA_JUNCHENG_8_HUMAN_10X-OJ1-cellranger-count-default/OJ1_cellranger_count_outs.tar...
Copying gs://asap-raw-data-team-sulzer/fastqs_nigra/OJ1/analysis/220224_OSAMA_JUNCHENG_8_HUMAN_10X-OJ1-cellranger-count-default/README.txt [Content-Type=text/plain]...
Removing gs://asap-raw-data-team-sulzer/fastqs_nigra/OJ1/analysis/220224_OSAMA_JUNCHENG_8_HUMAN_10X-OJ1-cellranger

In [350]:

!gsutil -u dnastack-asap-parkinsons mv "gs://asap-raw-data-team-sulzer/fastqs/cingulate/**/*.fastq.gz" "gs://asap-raw-data-team-sulzer/fastqs/cingulate/"


Copying gs://asap-raw-data-team-sulzer/fastqs/cingulate/OF001/fastq/SN_3PGEX_10X/210210_A00521_0199_BHNMK3DSXY/OF001_S1_L001_I1_001.fastq.gz [Content-Type=application/octet-stream]...
Removing gs://asap-raw-data-team-sulzer/fastqs/cingulate/OF001/fastq/SN_3PGEX_10X/210210_A00521_0199_BHNMK3DSXY/OF001_S1_L001_I1_001.fastq.gz...
Copying gs://asap-raw-data-team-sulzer/fastqs/cingulate/OF001/fastq/SN_3PGEX_10X/210210_A00521_0199_BHNMK3DSXY/OF001_S1_L001_I2_001.fastq.gz [Content-Type=application/octet-stream]...
Removing gs://asap-raw-data-team-sulzer/fastqs/cingulate/OF001/fastq/SN_3PGEX_10X/210210_A00521_0199_BHNMK3DSXY/OF001_S1_L001_I2_001.fastq.gz...
Copying gs://asap-raw-data-team-sulzer/fastqs/cingulate/OF001/fastq/SN_3PGEX_10X/210210_A00521_0199_BHNMK3DSXY/OF001_S1_L001_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Removing gs://asap-raw-data-team-sulzer/fastqs/cingulate/OF001/fastq/SN_3PGEX_10X/210210_A00521_0199_BHNMK3DSXY/OF001_S1_L001_R1_001.fastq.gz...
Copying gs://

In [353]:
!gsutil -u dnastack-asap-parkinsons ls -al "gs://asap-raw-data-team-sulzer/fastqs/nigra"


 342559148  2023-12-06T00:19:01Z  gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ10_S1_L001_I1_001.fastq.gz#1701821941986375  metageneration=1
 359917684  2023-12-06T00:19:02Z  gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ10_S1_L001_I2_001.fastq.gz#1701821942302756  metageneration=1
3431481573  2023-12-06T00:19:02Z  gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ10_S1_L001_R1_001.fastq.gz#1701821942601943  metageneration=1
2572117362  2023-12-06T00:19:02Z  gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ10_S1_L001_R2_001.fastq.gz#1701821942905242  metageneration=1
 341079281  2023-12-06T00:19:03Z  gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ10_S1_L002_I1_001.fastq.gz#1701821943223278  metageneration=1
 359720996  2023-12-06T00:19:03Z  gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ10_S1_L002_I2_001.fastq.gz#1701821943511357  metageneration=1
3410135611  2023-12-06T00:19:03Z  gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ10_S1_L002_R1_001.fastq.gz#1701821943811147  metageneration=1
255816

In [352]:

!gsutil -u dnastack-asap-parkinsons mv "gs://asap-raw-data-team-sulzer/fastqs/nigra/**/*.fastq.gz" "gs://asap-raw-data-team-sulzer/fastqs/nigra/"


Copying gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ1/fastq/SC_3PGEX_10X/220225_A00521_0295_AHMW3YDSX2/OJ1_S1_L001_I1_001.fastq.gz [Content-Type=application/octet-stream]...
Removing gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ1/fastq/SC_3PGEX_10X/220225_A00521_0295_AHMW3YDSX2/OJ1_S1_L001_I1_001.fastq.gz...
Copying gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ1/fastq/SC_3PGEX_10X/220225_A00521_0295_AHMW3YDSX2/OJ1_S1_L001_I2_001.fastq.gz [Content-Type=application/octet-stream]...
Removing gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ1/fastq/SC_3PGEX_10X/220225_A00521_0295_AHMW3YDSX2/OJ1_S1_L001_I2_001.fastq.gz...
Copying gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ1/fastq/SC_3PGEX_10X/220225_A00521_0295_AHMW3YDSX2/OJ1_S1_L001_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Removing gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ1/fastq/SC_3PGEX_10X/220225_A00521_0295_AHMW3YDSX2/OJ1_S1_L001_R1_001.fastq.gz...
Copying gs://asap-raw-data-team-sulzer/fastqs/nigra/OJ1/fastq

In [341]:
!gsutil -u dnastack-asap-parkinsons mv "gs://asap-raw-data-team-sulzer/fastqs/cingulate/bam_files" "gs://asap-raw-data-team-sulzer/artifacts/bam_files"


Copying gs://asap-raw-data-team-sulzer/fastqs/cingulate/bam_files/OF001/index.html [Content-Type=text/html]...
Removing gs://asap-raw-data-team-sulzer/fastqs/cingulate/bam_files/OF001/index.html...
Copying gs://asap-raw-data-team-sulzer/fastqs/cingulate/bam_files/OF001/possorted_genome_bam.bam [Content-Type=application/octet-stream]...
Removing gs://asap-raw-data-team-sulzer/fastqs/cingulate/bam_files/OF001/possorted_genome_bam.bam...
Copying gs://asap-raw-data-team-sulzer/fastqs/cingulate/bam_files/OF002/index.html [Content-Type=text/html]...
Removing gs://asap-raw-data-team-sulzer/fastqs/cingulate/bam_files/OF002/index.html...
Copying gs://asap-raw-data-team-sulzer/fastqs/cingulate/bam_files/OF002/possorted_genome_bam.bam [Content-Type=application/octet-stream]...
Removing gs://asap-raw-data-team-sulzer/fastqs/cingulate/bam_files/OF002/possorted_genome_bam.bam...

==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil 

In [335]:
!gsutil -u dnastack-asap-parkinsons mv "gs://asap-raw-data-team-sulzer/fastqs/**/analysis/*tar*" "gs://asap-raw-data-team-sulzer/artifacts/cellranger_counts"


Copying gs://asap-raw-data-team-sulzer/fastqs/cingulate/OF001/analysis/201123_OSAMA_FATIMA_2_HUMAN_10X-OF001-cellranger-count-default/OF001_cellranger_count_info.tar.gz [Content-Type=application/x-tar]...
Removing gs://asap-raw-data-team-sulzer/fastqs/cingulate/OF001/analysis/201123_OSAMA_FATIMA_2_HUMAN_10X-OF001-cellranger-count-default/OF001_cellranger_count_info.tar.gz...
Copying gs://asap-raw-data-team-sulzer/fastqs/cingulate/OF001/analysis/201123_OSAMA_FATIMA_2_HUMAN_10X-OF001-cellranger-count-default/OF001_cellranger_count_outs.tar [Content-Type=application/x-tar]...
Removing gs://asap-raw-data-team-sulzer/fastqs/cingulate/OF001/analysis/201123_OSAMA_FATIMA_2_HUMAN_10X-OF001-cellranger-count-default/OF001_cellranger_count_outs.tar...
Copying gs://asap-raw-data-team-sulzer/fastqs/cingulate/OF002/analysis/201123_OSAMA_FATIMA_2_HUMAN_10X-OF002-cellranger-count-default/OF002_cellranger_count_info.tar.gz [Content-Type=application/x-tar]...
Removing gs://asap-raw-data-team-sulzer/fastq

## get artifacts

 "gs://asap-raw-data-team-sulzer/manifest.txt"

In [90]:
!gsutil -u dnastack-asap-parkinsons -m cp -r \
  "gs://asap-raw-data-team-sulzer/artifacts" \
  /Users/ergonyc/Projects/ASAP/team-sulzer/

If you experience problems with multiprocessing on MacOS, they might be related to https://bugs.python.org/issue33725. You can disable multiprocessing by editing your .boto config or by adding the following flag to your command: `-o "GSUtil:parallel_process_count=1"`. Note that multithreading is still available even if you disable multiprocessing.

Copying gs://asap-raw-data-team-sulzer/artifacts/cingulate_lineages_sublineages_assigned_dims18_k20_res02_final.rds...
==> NOTE: You are downloading one or more large file(s), which would            
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

Copying gs://asap-raw-data-team-sulzer/artifacts/master_seurat_PD_filtered_doublet_removed_SCT_clusterslm3_res01_linege_called.rds...
Copying gs://asap-raw-data-team-sulzer/artifacts/nigra_cellbender.rds...        
Copying gs://asap-raw-data-team-sulzer/artifacts/nigra_obj

In [91]:
!gsutil -u dnastack-asap-parkinsons -m cp -r \
  "gs://asap-raw-data-team-sulzer/scripts" \
  /Users/ergonyc/Projects/ASAP/team-sulzer/

If you experience problems with multiprocessing on MacOS, they might be related to https://bugs.python.org/issue33725. You can disable multiprocessing by editing your .boto config or by adding the following flag to your command: `-o "GSUtil:parallel_process_count=1"`. Note that multithreading is still available even if you disable multiprocessing.

Copying gs://asap-raw-data-team-sulzer/scripts/cellbender_sample_processing.R...
/ [1/1 files][  1.9 KiB/  1.9 KiB] 100% Done                                    
Operation completed over 1 objects/1.9 KiB.                                      


In [ ]:
gsutil -m cp \
  "gs://asap-raw-data-team-sulzer/artifacts/cingulate_lineages_sublineages_assigned_dims18_k20_res02_final.rds" \
  "gs://asap-raw-data-team-sulzer/artifacts/master_seurat_PD_filtered_doublet_removed_SCT_clusterslm3_res01_linege_called.rds" \
  "gs://asap-raw-data-team-sulzer/artifacts/nigra_cellbender.rds" \
  "gs://asap-raw-data-team-sulzer/artifacts/nigra_obj_dims16_res02_k20_sct_harmony_dblrm_processed_final.rds" \
  .